## Requirements
- `astropy.io.fits` to access the fits files
- `matplotlib.pyplot` for plotting
- `numpy` for array handling
- `os` for file directory finding

In [1]:
from astropy.io import fits
import matplotlib.pyplot as plt
import numpy as np
import os
%matplotlib qt

# ONLY WORKS ON MY LAPTOP, COULDN'T GET THE PATH TO WORK FOR THE REMOTE REPO
direc = r"C:\Users\rober\OneDrive\Documents\_Docs\Year 4\TGP\TGP_Asteroids2_2025"

 Useful `.header` keys:
- RA
- DEC 
- AZIMUTH
- ALTITUDE
- FILTER 
- EXPOSURE

## Cropping
- open and display first image from night 1 to see the border, and crop until the border disappears

In [2]:
# READ FIRST IMAGE
image = fits.open(direc + r'\2025-10-09\PIRATE_165659_OSL_ROE_Asteroids_2_00_Standard_Star_1_00_Filter_R_00_2025_10_09_19_46_12.fits')[0].data

# REMOVE BORDER
image = image[21:2027,21:2027]

# PLOT
fig,ax = plt.subplots(figsize=(8,8))
ax.imshow(image, cmap='gray', vmax=1500)
plt.show()

del image, fig, ax

- Crop all files from all nights, and all calibration frames

In [3]:
# EXTRACT ALL SCIENCE FRAMES
fnames_day1 = os.listdir(direc + r'\2025-10-09')
files_day1 = [direc + '\\2025-10-09\\' + file for file in fnames_day1]
fnames_day2 = os.listdir(direc + r'\2025-10-19')
files_day2 = [direc + '\\2025-10-19\\' + file for file in fnames_day2]
fnames_day3 = os.listdir(direc + r'\2025-10-22')
files_day3 = [direc + '\\2025-10-22\\' + file for file in fnames_day3]

# EXTRACT ALL CALIBRATION FRAMES
cnames_day1 = os.listdir(direc + r'\Calibrations_1')
calibs_day1 = [direc + '\\Calibrations_1\\' + file for file in cnames_day1]
cnames_day2 = os.listdir(direc + r'\Calibrations_2')
calibs_day2 = [direc + '\\Calibrations_2\\' + file for file in cnames_day2]
cnames_day3 = os.listdir(direc + r'\Calibrations_3')
calibs_day3 = [direc + '\\Calibrations_3\\' + file for file in cnames_day3]

# COMBINE ALL FRAMES
all_names = fnames_day1 + fnames_day2 + fnames_day3 + cnames_day1 + cnames_day2 + cnames_day3
all_files = files_day1 + files_day2 + files_day3 + calibs_day1 + calibs_day2 + calibs_day3

# CROP ALL AND GIVE KEY AS EXPOSURE CODE (E.G. 162431), SHAPES WILL BE (2006,2006)
cropped = {}
headers = {}
# for (name, file) in zip(all_names, all_files):
#     cropped[name[7:13]] = fits.open(file)[0].data[21:2027,21:2027]
#     headers[name[7:13]] = fits.open(file)[0].header
for (name, file) in zip(all_names, all_files):
    with fits.open(file) as datafile:
        cropped[name[7:13]] = datafile[0].data[21:2027,21:2027]
        headers[name[7:13]] = datafile[0].header

# CHECK ALL CROPPED IMAGES HAVE THE CORRECT SHAPE
for key in cropped:
    if cropped[key].shape != (2006,2006):
        print(key)

del files_day1, files_day2, files_day3
del cnames_day1, cnames_day2, cnames_day3
del calibs_day1, calibs_day2, calibs_day3
del all_names, all_files

## Make bias frames

In [4]:
# GET MEDIAN BIAS FRAME FOR EACH NIGHT
bias_day1 = np.median(np.stack([cropped["164291"],
                                cropped["164292"],
                                cropped["164293"],
                                cropped["164294"]]), axis=0)
bias_day2 = np.median(np.stack([cropped["165895"],
                                cropped["165896"],
                                cropped["165897"],
                                cropped["165898"]]), axis=0)
bias_day3 = np.median(np.stack([cropped["166960"],
                                cropped["166961"],
                                cropped["166962"],
                                cropped["166963"]]), axis=0)

## Make dark frames

In [5]:
# BIAS-SUBTRACT THE DARK FRAMES
dark1_bs_60s = np.median(np.stack([cropped["164277"] - bias_day1,
                                   cropped["164278"] - bias_day1,
                                   cropped["164279"] - bias_day1,
                                   cropped["164280"] - bias_day1]), axis=0)
dark2_bs_60s = np.median(np.stack([cropped["165881"] - bias_day2,
                                   cropped["165882"] - bias_day2,
                                   cropped["165883"] - bias_day2,
                                   cropped["165884"] - bias_day2]), axis=0)
dark3_bs_60s = np.median(np.stack([cropped["166946"] - bias_day3,
                                   cropped["166947"] - bias_day3,
                                   cropped["166948"] - bias_day3,
                                   cropped["166949"] - bias_day3]), axis=0)

## Make all flat frames

In [6]:
# EXTRACT ALL FLAT CODES
flat_codes1 = [str(i) for i in range(162424,162440) if i not in (162428,162429,162434,162435)] #4x BVR (12)
flat_codes1 += ["163459", "163461", "163462"] + [str(i) for i in range(164325, 164330)] # 8x I
flat_codes2 = [str(i) for i in range(165642,165658) if i not in (165646,165647,165652,165653)] #4x BVR
flat_codes3 = [str(i) for i in range(166095,166111) if i not in (166099,166100,166105,166106)] #4x BVR
print("codes extracted")

# BIAS SUBTRACT THE FLATS
flats1_bs = {}
for c in flat_codes1:
    flats1_bs[c] = cropped[c] - bias_day1
print("bias 1")
flats2_bs = {}
for c in flat_codes2:
    flats2_bs[c] = cropped[c] - bias_day2
print("bias 2")
flats3_bs = {}
for c in flat_codes3:
    flats3_bs[c] = cropped[c] - bias_day3
print("bias 3")

# SCALE DARK AND SUBTRACT
flats1_bs_ds = {}
for c in flat_codes1:
    flats1_bs_ds[c] = flats1_bs[c] - dark1_bs_60s * (headers[c]["EXPOSURE"]/60)
print("dark 1")
flats2_bs_ds = {}
for c in flat_codes2:
    flats2_bs_ds[c] = flats2_bs[c] - dark2_bs_60s * (headers[c]["EXPOSURE"]/60)
print("dark 2")
flats3_bs_ds = {}
for c in flat_codes3:
    flats3_bs_ds[c] = flats3_bs[c] - dark3_bs_60s * (headers[c]["EXPOSURE"]/60)
print("dark 3")

# NORMALISE, STACK AND RE-NORMALISE
flat_B1 = np.median(np.stack([flats1_bs_ds["162424"]/np.mean(flats1_bs_ds["162424"]),
                              flats1_bs_ds["162425"]/np.mean(flats1_bs_ds["162425"]),
                              flats1_bs_ds["162426"]/np.mean(flats1_bs_ds["162426"]),
                              flats1_bs_ds["162427"]/np.mean(flats1_bs_ds["162427"])]), axis=0)
flat_B1 /= np.mean(flat_B1)
flat_V1 = np.median(np.stack([flats1_bs_ds["162430"]/np.mean(flats1_bs_ds["162430"]),
                              flats1_bs_ds["162431"]/np.mean(flats1_bs_ds["162431"]),
                              flats1_bs_ds["162432"]/np.mean(flats1_bs_ds["162432"]),
                              flats1_bs_ds["162433"]/np.mean(flats1_bs_ds["162433"])]), axis=0)
flat_V1 /= np.mean(flat_V1)
flat_R1 = np.median(np.stack([flats1_bs_ds["162436"]/np.mean(flats1_bs_ds["162436"]),
                              flats1_bs_ds["162437"]/np.mean(flats1_bs_ds["162437"]),
                              flats1_bs_ds["162438"]/np.mean(flats1_bs_ds["162438"]),
                              flats1_bs_ds["162439"]/np.mean(flats1_bs_ds["162439"])]), axis=0)
flat_R1 /= np.mean(flat_R1)
# flat_I1 ??
print("normalise 1")

flat_B2 = np.median(np.stack([flats2_bs_ds["165642"]/np.mean(flats2_bs_ds["165642"]),
                              flats2_bs_ds["165643"]/np.mean(flats2_bs_ds["165643"]),
                              flats2_bs_ds["165644"]/np.mean(flats2_bs_ds["165644"]),
                              flats2_bs_ds["165645"]/np.mean(flats2_bs_ds["165645"])]), axis=0)
flat_B2 /= np.mean(flat_B2)
flat_V2 = np.median(np.stack([flats2_bs_ds["165648"]/np.mean(flats2_bs_ds["165648"]),
                              flats2_bs_ds["165649"]/np.mean(flats2_bs_ds["165649"]),
                              flats2_bs_ds["165650"]/np.mean(flats2_bs_ds["165650"]),
                              flats2_bs_ds["165651"]/np.mean(flats2_bs_ds["165651"])]), axis=0)
flat_V2 /= np.mean(flat_V2)
flat_R2 = np.median(np.stack([flats2_bs_ds["165654"]/np.mean(flats2_bs_ds["165654"]),
                              flats2_bs_ds["165655"]/np.mean(flats2_bs_ds["165655"]),
                              flats2_bs_ds["165656"]/np.mean(flats2_bs_ds["165656"]),
                              flats2_bs_ds["165657"]/np.mean(flats2_bs_ds["165657"])]), axis=0)
flat_R2 /= np.mean(flat_R2)
print("normalise 2")

flat_B3 = np.median(np.stack([flats3_bs_ds["166095"]/np.mean(flats3_bs_ds["166095"]),
                              flats3_bs_ds["166096"]/np.mean(flats3_bs_ds["166096"]),
                              flats3_bs_ds["166097"]/np.mean(flats3_bs_ds["166097"]),
                              flats3_bs_ds["166098"]/np.mean(flats3_bs_ds["166098"])]), axis=0)
flat_B3 /= np.mean(flat_B3)
flat_V3 = np.median(np.stack([flats3_bs_ds["166101"]/np.mean(flats3_bs_ds["166101"]),
                              flats3_bs_ds["166102"]/np.mean(flats3_bs_ds["166102"]),
                              flats3_bs_ds["166103"]/np.mean(flats3_bs_ds["166103"]),
                              flats3_bs_ds["166104"]/np.mean(flats3_bs_ds["166104"])]), axis=0)
flat_V3 /= np.mean(flat_V3)
flat_R3 = np.median(np.stack([flats3_bs_ds["166107"]/np.mean(flats3_bs_ds["166107"]),
                              flats3_bs_ds["166108"]/np.mean(flats3_bs_ds["166108"]),
                              flats3_bs_ds["166109"]/np.mean(flats3_bs_ds["166109"]),
                              flats3_bs_ds["166110"]/np.mean(flats3_bs_ds["166110"])]), axis=0)
flat_R3 /= np.mean(flat_R3)
print("normalise 3")

# CHECK ALL FRAMES ARE PROPERLY NORMALISED
# print(np.mean(flat_B1))
# print(np.mean(flat_V1))
# print(np.mean(flat_R1))
# print(np.mean(flat_B2))
# print(np.mean(flat_V2))
# print(np.mean(flat_R2))
# print(np.mean(flat_B3))
# print(np.mean(flat_V3))
# print(np.mean(flat_R3))

del flat_codes1, flat_codes2, flat_codes3
del flats1_bs, flats2_bs, flats3_bs
del flats1_bs_ds, flats2_bs_ds, flats3_bs_ds

codes extracted
bias 1
bias 2
bias 3
dark 1
dark 2
dark 3
normalise 1
normalise 2
normalise 3


In [7]:
#  PLOT ALL FLATS
fig,axs = plt.subplots(3,3,figsize=(12,12))
axs[0,0].imshow(flat_B1, cmap="plasma", vmin=.94, vmax=1.06)
axs[0,0].set_title("B1")
axs[0,1].imshow(flat_V1, cmap="plasma", vmin=.94, vmax=1.06)
axs[0,1].set_title("V1")
axs[0,2].imshow(flat_R1, cmap="plasma", vmin=.94, vmax=1.06)
axs[0,2].set_title("R1")
axs[1,0].imshow(flat_B2, cmap="plasma", vmin=.94, vmax=1.06)
axs[1,1].imshow(flat_V2, cmap="plasma", vmin=.94, vmax=1.06)
axs[1,2].imshow(flat_R2, cmap="plasma", vmin=.94, vmax=1.06)
axs[2,0].imshow(flat_B3, cmap="plasma", vmin=.94, vmax=1.06)
axs[2,1].imshow(flat_V3, cmap="plasma", vmin=.94, vmax=1.06)
axs[2,2].imshow(flat_R3, cmap="plasma", vmin=.94, vmax=1.06)

fig.tight_layout(); plt.show()

del fig, axs

## Perform full science frame reduction

In [8]:
# EXTRACT NAMES, SPLIT INTO OBSERVATION DAYS
names1 = [full[7:13] for full in fnames_day1]
names2 = [full[7:13] for full in fnames_day2]
names3 = [full[7:13] for full in fnames_day3]

# BIAS SUBTRACT
frames_bs = {}
for n in names1:
    frames_bs[n] = cropped[n] - bias_day1
print("bias 1")
for n in names2:
    frames_bs[n] = cropped[n] - bias_day2
print("bias 2")
for n in names3:
    frames_bs[n] = cropped[n] - bias_day3
print("bias 3")

# SCALE DARK AND SUBTRACT
frames_bs_ds = {}
for n in names1:
    frames_bs_ds[n] = frames_bs[n] - dark1_bs_60s * (headers[n]["EXPOSURE"]/60)
print("Dark 1")
for n in names2:
    frames_bs_ds[n] = frames_bs[n] - dark2_bs_60s * (headers[n]["EXPOSURE"]/60)
print("Dark 2")
for n in names3:
    frames_bs_ds[n] = frames_bs[n] - dark3_bs_60s * (headers[n]["EXPOSURE"]/60)
print("Dark 3")
del frames_bs

# FLAT CORRECTION
final_frames = {}
for n in names1:
    if headers[n]["FILTER"] == "B":
        final_frames[n] = frames_bs_ds[n]/flat_B1
    elif headers[n]["FILTER"] == "V":
        final_frames[n] = frames_bs_ds[n]/flat_V1
    elif headers[n]["FILTER"] == "R":
        final_frames[n] = frames_bs_ds[n]/flat_R1
    else:
        print(n, headers[n]["FILTER"])
print("Flat 1")
for n in names2:
    if headers[n]["FILTER"] == "B":
        final_frames[n] = frames_bs_ds[n]/flat_B2
    elif headers[n]["FILTER"] == "V":
        final_frames[n] = frames_bs_ds[n]/flat_V2
    elif headers[n]["FILTER"] == "R":
        final_frames[n] = frames_bs_ds[n]/flat_R2
    else:
        print(n, headers[n]["FILTER"])
print("Flat 2")
for n in names3:
    if headers[n]["FILTER"] == "B":
        final_frames[n] = frames_bs_ds[n]/flat_B3
    elif headers[n]["FILTER"] == "V":
        final_frames[n] = frames_bs_ds[n]/flat_V3
    elif headers[n]["FILTER"] == "R":
        final_frames[n] = frames_bs_ds[n]/flat_R3
    else:
        print(n, headers[n]["FILTER"])
print("Flat 3")
del frames_bs_ds

bias 1
bias 2
bias 3
Dark 1
Dark 2
Dark 3
165677 I
165681 I
165685 I
165691 I
165692 I
165693 I
165706 I
165713 I
165714 I
165736 I
165743 I
165744 I
165745 I
Flat 1
167151 I
167153 I
167162 I
167168 I
Flat 2
167862 U
167866 I
167867 I
167868 I
167876 I
167882 I
167889 I
167890 I
167895 I
167897 I
167904 I
Flat 3


## Write to new `.fits` files

In [ ]:
# for n in names1:
#     try:
#         fits.PrimaryHDU(data=final_frames[n], header=headers[n]).writeto(n + ".fits")
#         print(f"Written: {n}")
#     except:
#         print(f"Not reduced: {n}")
    
# NOT REDUCED:
# 165677
# 165681
# 165685
# 165691
# 165692
# 165693
# 165706
# 165713
# 165714
# 165736
# 165743
# 165744
# 165745

Written: 165659
Written: 165660
Written: 165661
Written: 165662
Written: 165663
Written: 165674
Written: 165675
Written: 165676
Not reduced: 165677
Written: 165678
Written: 165679
Written: 165680
Not reduced: 165681
Written: 165682
Written: 165683
Written: 165684
Not reduced: 165685
Written: 165686
Written: 165687
Written: 165688
Written: 165689
Written: 165690
Not reduced: 165691
Not reduced: 165692
Not reduced: 165693
Written: 165694
Written: 165695
Written: 165696
Written: 165697
Written: 165698
Written: 165699
Written: 165700
Written: 165701
Written: 165703
Written: 165704
Written: 165705
Not reduced: 165706
Written: 165707
Written: 165708
Written: 165709
Written: 165710
Written: 165711
Written: 165712
Not reduced: 165713
Not reduced: 165714
Written: 165718
Written: 165719
Written: 165720
Written: 165731
Written: 165732
Written: 165733
Written: 165734
Written: 165735
Not reduced: 165736
Written: 165737
Written: 165738
Written: 165739
Written: 165740
Written: 165741
Written: 165742


In [ ]:
# for n in names2:
#     try:
#         fits.PrimaryHDU(data=final_frames[n], header=headers[n]).writeto(n + ".fits")
#         print(f"Written: {n}")
#     except:
#         print(f"Not reduced: {n}")

# NOT REDUCED:
# 167151
# 167153
# 167162
# 167168

Written: 167131
Written: 167133
Written: 167136
Written: 167137
Written: 167138
Written: 167139
Written: 167140
Written: 167141
Written: 167142
Written: 167143
Written: 167144
Written: 167147
Written: 167148
Written: 167149
Written: 167150
Not reduced: 167151
Written: 167152
Not reduced: 167153
Written: 167154
Written: 167155
Written: 167156
Written: 167157
Written: 167158
Written: 167159
Written: 167160
Written: 167161
Not reduced: 167162
Written: 167163
Written: 167164
Written: 167165
Written: 167166
Written: 167167
Not reduced: 167168
Written: 167169


In [ ]:
# for n in names3:
#     try:
#         fits.PrimaryHDU(data=final_frames[n], header=headers[n]).writeto(n + ".fits")
#         print(f"Written: {n}")
#     except:
#         print(f"Not reduced: {n}")

# NOT REDUCED:
# 167862
# 167866
# 167867
# 167868
# 167876
# 167882
# 167889
# 167890
# 167895
# 167897
# 167904


Written: 167819
Written: 167820
Written: 167821
Written: 167822
Written: 167823
Written: 167824
Written: 167825
Written: 167826
Written: 167827
Written: 167828
Written: 167829
Written: 167830
Written: 167831
Written: 167832
Written: 167833
Written: 167834
Written: 167835
Written: 167836
Written: 167837
Written: 167838
Written: 167839
Written: 167840
Written: 167841
Written: 167842
Written: 167843
Written: 167844
Written: 167845
Written: 167846
Written: 167851
Written: 167852
Written: 167853
Written: 167854
Written: 167855
Written: 167856
Written: 167857
Written: 167858
Written: 167859
Written: 167860
Written: 167861
Not reduced: 167862
Written: 167863
Written: 167864
Not reduced: 167866
Not reduced: 167867
Not reduced: 167868
Written: 167869
Written: 167870
Written: 167871
Written: 167872
Written: 167873
Written: 167874
Written: 167875
Not reduced: 167876
Written: 167877
Written: 167878
Written: 167879
Written: 167880
Written: 167881
Not reduced: 167882
Written: 167883
Written: 167884


In [88]:
# VIEWING WINDOW
with fits.open(direc + r"\Reduced_data_day1\165731.fits") as file:
    fig,ax = plt.subplots(figsize=(12,12))
    ax.imshow(file[0].data, cmap="plasma", vmin=800, vmax=1100)
    plt.show()